In [ ]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import os,re
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup

from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


data_set = tf.keras.utils.get_file(
    fname="imdb.tar.gz", #download file name
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract = True
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def directory_data(directory):
  data = {}
  data["review"] = []
  for file_path in os.listdir(directory):
    with open(os.path.join(directory, file_path), "r", encoding='utf-8') as file:
      data["review"].append(file.read())
  return pd.DataFrame.from_dict(data)

In [ ]:
def data(directory):
  pos_df = directory_data(os.path.join(directory, "pos"))
  neg_df = directory_data(os.path.join(directory, "neg"))
  pos_df["sentiment"] = 1
  neg_df["sentiment"] = 0
  return pd.concat([pos_df, neg_df])

In [ ]:
train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "train"))
test_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "test"))
imdb_pd = pd.concat([train_df, test_df])


In [ ]:
#알파벳 제외하고 날리기

def preprocessing(review, remove_stopwords = False):#불용어 없는 경우 if이하 작업 없이 그대로 출력
  review_text = BeautifulSoup(review, 'html5lib').get_text() #HTML 태그를 제거하고 텍스트만 뽑아냄
  review_text = re.sub("[^a-zA-Z]", " ",review)#영문자 이외의 모든 것을 공백 처리
  if remove_stopwords:# 불용어가 존재할 경우
    words = review_text.split()# 기존 텍스트를 모두 리스트 처리
    words = [ w for w in words if not w in stop_words]# 불용어가 아닌 문자들아 재 리스트화
    review_text = ' '.join(words)#리스트에 있는 단어들은 join으로 다시 결합
  return review_text

In [ ]:
reviews = list(['i am so happy'])
# reviews = list(imdb_pd['review'])


clean_reviews = []

for review in reviews:
  clean_reviews.append(preprocessing(review,True))
print(clean_reviews[0])



happy


In [ ]:
# clean 데이터 df로 만들기
clean_train_df = pd.DataFrame( { 'review': clean_reviews, 'sentiment' : imdb_pd['sentiment']})

reviews = list(clean_train_df['review'])

sentiment = list(clean_train_df['sentiment'])

In [ ]:
clean_train_df.head()

,review,sentiment
0,Trailers movie may show scenes violence non ma...,1
1,I sense mix confusion varying degrees personal...,1
2,Brando plays ace jet pilot back shooting MiGs ...,1
3,The planning episodes bit dull reached desert ...,1
4,A prequel imagined Battlestar Galactica series...,1


In [ ]:
list_reviews = list(clean_train_df['review'])
list_sentiments = list(clean_train_df['sentiment'])


vectorizer = TfidfVectorizer(min_df = 0.0, analyzer = 'char',
                            sublinear_tf =True, ngram_range = (1,3), max_features =5000)

X = vectorizer.fit_transform(list_reviews)
y = np.array(list_sentiments)
                            
                            

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model = Sequential()

model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = 'accuracy')


model.fit(X_train.toarray(), Y_train,epochs = 50, verbose = 1)

Epoch 1/50
1250/1250 [==============================] - 7s 4ms/step - loss: 0.6910 - accuracy: 0.5694
Epoch 2/50
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6871 - accuracy: 0.6794
Epoch 3/50
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6832 - accuracy: 0.7374
Epoch 4/50
1250/1250 [==============================] - 7s 6ms/step - loss: 0.6794 - accuracy: 0.7614
Epoch 5/50
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6756 - accuracy: 0.7749
Epoch 6/50
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6719 - accuracy: 0.7842
Epoch 7/50
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6683 - accuracy: 0.7861
Epoch 8/50
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6648 - accuracy: 0.7901
Epoch 9/50
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6613 - accuracy: 0.7949
Epoch 10/50
1250/1250 [==============================] - 4s 3ms/step - loss: 0.657

In [ ]:
_, accuracy = model.evaluate(X_test.toarray(), Y_test)
print('Accuracy: ', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.5609 - accuracy: 0.8157
Accuracy:  0.8156999945640564


In [ ]:
sentence = 'I dissatisfied about this movie. It was fun at first ten minutes, but not in else.'
clean_sen = preprocessing(sentence, True)
clean_sen

vec_sen = vectorizer.transform([clean_sen])
vec_sen

pred = model.predict(vec_sen.toarray())
result = np.where(pred >=0.5, 1, 0)
print(pred, result)

1/1 [==============================] - 0s 55ms/step
[[0.4782511]] [[0]]
